In [10]:
from nba_api.stats.static import teams
import pandas as pd
import time
pd.set_option('display.max_columns', None) 

/tmp/ipykernel_1190/1109419456.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)
/tmp/ipykernel_1190/1109419456.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)
/tmp/ipykernel_1190/1109419456.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or a

In [11]:
# Get all Team IDs
team_dict = teams.get_teams()
id_team_pairs = [(team['id'], team['full_name']) for team in team_dict]
team_ids = [pair[0] for pair in id_team_pairs]
#team_ids

In [12]:
# Get all team Stats
from nba_api.stats.endpoints import leaguegamefinder

all_teams_data_frames = []

# Loop through each team ID and fetch the corresponding team stats
for team_id in team_ids:
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    time.sleep(.600)
    team_data_frame = team_games.get_data_frames()[0]
    # team_data_frame = team_data_frame[team_data_frame['SEASON_ID'].str[1:] >= "2021"]
    all_teams_data_frames.append(team_data_frame)

In [13]:
all_teams_data_frames[0]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612737,ATL,Atlanta Hawks,0022300533,2024-01-12,ATL vs. IND,L,242,108,40,89,0.449,11,30,0.367,17,23,0.739,12,16,28,25,14.0,2,14,13,-18.0
1,22023,1610612737,ATL,Atlanta Hawks,0022300521,2024-01-10,ATL vs. PHI,W,265,139,48,102,0.471,12,34,0.353,31,42,0.738,16,39,55,32,6.0,8,12,21,7.0
2,22023,1610612737,ATL,Atlanta Hawks,0022300499,2024-01-07,ATL @ ORL,L,265,110,42,91,0.462,9,30,0.300,17,29,0.586,8,42,50,21,9.0,10,12,23,-7.0
3,22023,1610612737,ATL,Atlanta Hawks,0022300480,2024-01-05,ATL @ IND,L,238,116,40,93,0.430,10,42,0.238,26,30,0.867,14,25,39,21,6.0,3,14,11,-34.0
4,22023,1610612737,ATL,Atlanta Hawks,0022300467,2024-01-03,ATL vs. OKC,W,240,141,48,95,0.505,14,39,0.359,31,33,0.939,15,34,49,30,8.0,5,11,21,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,21983,1610612737,ATL,Atlanta Hawks,0028300058,1983-11-06,ATL @ MIL,L,240,84,32,78,0.410,0,2,0.000,20,30,0.667,11,26,37,18,4.0,5,14,24,NaN
3646,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,42,86,0.488,1,1,1.000,18,26,0.692,19,27,46,31,14.0,13,18,27,NaN
3647,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,38,81,0.469,0,0,NaN,19,30,0.633,12,29,41,20,7.0,10,16,34,NaN
3648,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,49,94,0.521,0,1,0.000,19,30,0.633,27,21,48,28,14.0,7,23,35,NaN


In [14]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.static import players
from nba_api.stats.endpoints import PlayerGameLogs

id = players.get_active_players()[3]["id"]

gamelog_lebron = PlayerGameLogs(season_nullable =['2021-22'])
gamelog_lebron_df = gamelog_lebron.get_data_frames()
#gamelog_lebron_df[0]

In [15]:
import datetime

player_id = players.get_active_players()[0]["id"]

# Get the current year
current_year = datetime.datetime.now().year

# Specify the range of seasons (2019 to the current year)
seasons_range = range(2019, current_year + 1)

# Fetch player game logs for each season and store in a list
all_gamelogs = [playergamelog.PlayerGameLog(player_id, season=season).get_data_frames()[0] for season in seasons_range]

# Concatenate all game logs into a single DataFrame
combined_gamelogs_df = pd.concat(all_gamelogs, ignore_index=True)
#combined_gamelogs_df

/tmp/ipykernel_1190/1680663178.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(all_gamelogs, ignore_index=True)


In [37]:
from nba_api.stats.static import players

player_dict = players.get_active_players()
id_player_pairs = dict([(player['id'], player['full_name']) for player in player_dict])
player_ids = [pair for pair in id_player_pairs]
id_player_pairs.get(2544)

'LeBron James'

In [46]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll

# Get the current year & Specify the range of seasons (2019 to the current year)
current_year = datetime.datetime.now().year
seasons_range = range(2019, current_year + 1)

# Loop through each player ID and fetch the corresponding team stats
all_players_data_frames = []
for player_id in player_ids:
    player_gamelog = [playergamelog.PlayerGameLog(player_id, season=season).get_data_frames()[0] for season in seasons_range]
    combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)
    combined_gamelogs_df.insert(2, 'Player_Name', combined_gamelogs_df['Player_ID'].map(id_player_pairs))
    #time.sleep(.6)
    all_players_data_frames.append(combined_gamelogs_df)
    print(len(all_players_data_frames))

/tmp/ipykernel_1190/2764691618.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)


1


/tmp/ipykernel_1190/2764691618.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)


2


/tmp/ipykernel_1190/2764691618.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)


...

/tmp/ipykernel_1190/2764691618.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)


529


/tmp/ipykernel_1190/2764691618.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)


530
531


/tmp/ipykernel_1190/2764691618.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)


In [50]:
all_players_data_frames[1]

,SEASON_ID,Player_ID,Player_Name,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22019,203500,Steven Adams,0021901317,"AUG 14, 2020",OKC @ LAC,L,6,0,2,0.000,0,0,0.0,0,0,0.000,2,2,4,0,0,0,1,0,0,2,1
1,22019,203500,Steven Adams,0021901306,"AUG 12, 2020",OKC vs. MIA,W,20,4,7,0.571,0,0,0.0,0,2,0.000,2,6,8,0,1,0,0,0,8,-7,1
2,22019,203500,Steven Adams,0021901265,"AUG 05, 2020",OKC @ LAL,W,28,7,10,0.700,0,0,0.0,4,7,0.571,1,6,7,2,0,0,1,2,18,13,1
3,22019,203500,Steven Adams,0021901251,"AUG 03, 2020",OKC vs. DEN,L,34,3,6,0.500,0,0,0.0,3,4,0.750,3,7,10,1,0,0,2,6,9,-18,1
4,22019,203500,Steven Adams,0021901240,"AUG 01, 2020",OKC vs. UTA,W,28,7,10,0.700,0,0,0.0,2,5,0.400,3,8,11,2,0,2,4,3,16,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,22022,203500,Steven Adams,0022200069,"OCT 27, 2022",MEM @ SAC,W,24,6,9,0.667,0,0,0.0,0,3,0.000,7,4,11,4,0,0,4,4,12,2,1
235,22022,203500,Steven Adams,0022200049,"OCT 24, 2022",MEM vs. BKN,W,31,3,5,0.600,0,0,0.0,3,6,0.500,4,9,13,2,1,1,0,2,9,20,1
236,22022,203500,Steven Adams,0022200034,"OCT 22, 2022",MEM @ DAL,L,22,1,3,0.333,0,0,0.0,0,2,0.000,1,3,4,1,1,4,3,1,2,-30,1
237,22022,203500,Steven Adams,0022200024,"OCT 21, 2022",MEM @ HOU,W,33,3,4,0.750,0,0,0.0,0,2,0.000,2,7,9,2,0,2,0,5,6,23,1


In [83]:
from concurrent.futures import ThreadPoolExecutor
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
from retrying import retry

@retry(stop_max_attempt_number=3, wait_fixed=2000)

def fetch_player_data(player_id):
    player_gamelog = [playergamelog.PlayerGameLog(player_id, season=season).get_data_frames()[0] for season in seasons_range]
    combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)
    time.sleep(1)
    return combined_gamelogs_df

with ThreadPoolExecutor() as executor:
    all_players_data_frames = list(executor.map(fetch_player_data, player_ids))

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [8]:
from concurrent.futures import ThreadPoolExecutor
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
import concurrent, time, datetime

# Get the current year & Specify the range of seasons (2019 to the current year)
current_year = datetime.datetime.now().year
seasons_range = range(2019, current_year + 1)

# Function to fetch player game logs for a given player ID
def fetch_player_gamelog(player_id):
    player_gamelog = [playergamelog.PlayerGameLog(player_id, season=season).get_data_frames()[0] for season in seasons_range]
    combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)
    return combined_gamelogs_df

# Using ThreadPoolExecutor for concurrent execution
all_players_data_frames = []
with ThreadPoolExecutor() as executor:
    # Fetch player game logs concurrently
    futures = [executor.submit(fetch_player_gamelog, player_id) for player_id in player_ids]

    # Collect the results as they become available
    for future in concurrent.futures.as_completed(futures):
        player_gamelog_df = future.result()
        all_players_data_frames.append(player_gamelog_df)
        time.sleep(.600)
        print(len(all_players_data_frames))

/tmp/ipykernel_1190/1109419456.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)
/tmp/ipykernel_1190/1109419456.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gamelogs_df = pd.concat(player_gamelog, ignore_index=True)
/tmp/ipykernel_1190/1109419456.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or a

1
2
3


KeyboardInterrupt: 